In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scikit_posthocs as sp
import seaborn as sns
import matplotlib as plt

In [ ]:
df = pd.read_csv("../03_datasets/frailty_ntpro_analysis.csv")

In [ ]:
for col in df.columns:
    if col.startswith("NTproBNP_Result "):
        df['NTproBNP_Result_1'] = df['NTproBNP_Result 1'].replace({'<50': 50,
                                                                   '>35000': 35000,
                                                                   '>70000': 70000})


df['NTproBNP_Result_1'] = pd.to_numeric(df['NTproBNP_Result_1'], errors='coerce')
df['Cg_NT_pro_BNP_Result_ng_L_'] = pd.to_numeric(df['Cg_NT_pro_BNP_Result_ng_L_'], errors='coerce')


In [ ]:
df['Admission_Date'] = pd.to_datetime(df['Admission_Date'])
df['Discharge_Date'] = pd.to_datetime(df['Discharge_Date'])

In [ ]:
# NT-proBNP results from  Healtheintent database are all upon admission
mask = df['Cg_NT_pro_BNP_Result_ng_L_'] == df['NTproBNP_Result_1']
df.loc[mask, 'NTproBNP_DT 1'] = df['Admission_Date']


In [ ]:
for col in df.columns:
    if col.startswith("NTproBNP_DT "):
        df[col] = pd.to_datetime(df[col], format='%d/%m/%Y')
        df[f"Admission_{col}"] = (df[col] - df['Admission_Date']).dt.days

In [ ]:
for col in df.columns:
    if col.startswith("NTproBNP_DT "):
        df[f"Discharge_{col}"] = (df['Discharge_Date'] - df[col]).dt.days

In [ ]:
# Get patients with NT-proBNP within first 72 hr of admission
num_NTproBNP = sum(col.startswith("Admission_NTproBNP_DT ") for col in df.columns)
df['NTpro_72h_admission'] = None

for idx, row in df.iterrows():
    for n in range(1, num_NTproBNP): 
        dt_col = f'Admission_NTproBNP_DT {n}'
        result_col = f'NTproBNP_Result {n}'
        if dt_col in df.columns and result_col in df.columns:
            if row[dt_col] <= 3:
                df.at[idx, 'NTpro_72h_admission'] = row[result_col]
                break  

In [ ]:
# Get patients with NT-proBNP within last 72 hr of admission
num_NTproBNP = sum(col.startswith("Discharge_NTproBNP_DT ") for col in df.columns)
df['NTpro_72h_discharge'] = None

for idx, row in df.iterrows():
    for n in range(num_NTproBNP, 0, -1):  # Count backwards
        dt_col = f'Discharge_NTproBNP_DT {n}'
        result_col = f'NTproBNP_Result {n}'
        if dt_col in df.columns and result_col in df.columns:
            if row[dt_col] <= 3:
                df.at[idx, 'NTpro_72h_discharge'] = row[result_col]
                break  

In [ ]:
# Only want analysis on patients with NT-proBNP within first and last 72hrs - not applicable otherwise
df['NTpro_72h_admission'] = pd.to_numeric(df['NTpro_72h_admission'], errors='coerce')
df['NTpro_72h_discharge'] = pd.to_numeric(df['NTpro_72h_discharge'], errors='coerce')

mask = (df['NTpro_72h_admission'] != df['NTpro_72h_discharge']) & df['NTpro_72h_discharge'].notna() & df['NTpro_72h_admission'].notna()
working_df = df.loc[mask]


In [ ]:
# How much has NT-proBNP changed (% of admission value) over admission
working_df['pc_change_72h'] = ((working_df['NTpro_72h_discharge'] - working_df['NTpro_72h_admission'])/working_df['NTpro_72h_admission']) *100

In [ ]:
working_df['pc_change_72h'].describe()

In [ ]:
# Remove positive extremes - Ameet email
working_df_cut = working_df[(working_df['pc_change_72h'] <= 100) & (working_df['pc_change_72h'] >= -100)]

In [ ]:
# Look at only those who have improved
df_working_neg = working_df[working_df['pc_change_72h'] < 0]
fig = px.histogram(df_working_neg, x='pc_change_72h', nbins=50)
fig.update_layout(
    xaxis_title="NTproBNP % Change Admission to Discharge", 
    yaxis_title="Frequency", 
    bargap=0.2,
    xaxis=dict(range=[-100, 0])  
)

fig.show()


In [ ]:
df_working_neg['pc_change_72h'].describe()

In [ ]:
# All outcomes (removed positive extremes)
outcomes = ['Died_within_30days_of_Discharge', 'Am_Died_within_90days_of_Discharge', 'An_Died_within_365days_of_Discharge', 
            'Readmission7d', 'Readmission14d', 'Readmission30d', 'Readmission180d']

sig_outcomes = []

for outcome in outcomes:
    outcome_1 = working_df_cut[working_df_cut[outcome] == 1]
    outcome_0 = working_df_cut[working_df_cut[outcome] == 0]
    
    # Histogram of %NT-pro change per outcome 
    hist_1 = go.Histogram(
        x=outcome_1['pc_change_72h'],
        nbinsx=50,
        name=f"{outcome} = 1",
        xaxis='x1',
        yaxis='y1'
    )
    
    hist_0 = go.Histogram(
        x=outcome_0['pc_change_72h'],
        nbinsx=50,
        name=f"{outcome} = 0",
        xaxis='x2',
        yaxis='y2'
    )
    
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.2,
        subplot_titles=[f"{outcome} = 1", f"{outcome} = 0"]
    )
    
    
    fig.add_trace(hist_1, row=1, col=1)
    fig.add_trace(hist_0, row=2, col=1)
    
    
    fig.update_layout(
        xaxis_title="pc_change_72h",
        yaxis_title="Frequency",
        bargap=0.2,
        xaxis=dict(range=[-100, 100]), 
        showlegend=False,
        height=600,
    )
    
    fig.show()
    
    group_1 = outcome_1['pc_change_72h']
    group_0 = outcome_0['pc_change_72h']
    
    # Mann-Whitney U Test (non-parametric) - differences between % change on outcome
    mannwhitney_u_stat, mannwhitney_u_p_value = stats.mannwhitneyu(group_1, group_0)

    print(f"Mann-Whitney U Test p-value: {mannwhitney_u_p_value}")

    if mannwhitney_u_p_value < 0.05:
        sig_outcomes.append(outcome)

In [ ]:
# Outcomes with signifiant differences between % change in NT-proBNP during admission
print(sig_outcomes)

In [ ]:
# Only look at outcomes with statistically significant differences - all outcomes (removed positive extremes)
for outcome in sig_outcomes:

    outcome_1 = working_df_cut[working_df_cut[outcome] == 1]
    outcome_0 = working_df_cut[working_df_cut[outcome] == 0]

    group_1 = outcome_1['pc_change_72h']
    group_0 = outcome_0['pc_change_72h']
    
    # Median change in NT-pro per group (outcome = 1/0)
    median_1 = group_1.median()
    median_0 = group_0.median()
    
    # Mann-Whitney
    mannwhitney_u_stat, mannwhitney_u_p_value = stats.mannwhitneyu(group_1, group_0)
    
    # Effect size
    n_1 = len(group_1)
    n_0 = len(group_0)
    rank_biserial_corr = (2 * mannwhitney_u_stat) / (n_1 * n_0) - 1
    
    # Violin plots
    combined_data = pd.concat([group_1, group_0], axis=0)
    labels = [f'{outcome} = 1'] * len(group_1) + [f'{outcome} = 0'] * len(group_0)
    
    fig_violin = px.violin(
        x=combined_data,
        y=labels,
        box=True,
        points="all",
        labels={"x": "pc_change_72h", "y": f"{outcome}"},
    )
    
    print(f"Median of pc_change_72h for {outcome} = 1: {median_1}")
    print(f"Median of pc_change_72h for {outcome} = 0: {median_0}")
    print(f"Mann-Whitney U Test p-value: {mannwhitney_u_p_value}")
    print(f"Rank-biserial correlation: {rank_biserial_corr}")
    
    fig_violin.show()


In [ ]:
# Same analysis, only for those that have improved
df_working_neg = working_df[working_df['pc_change_72h'] < 0]

outcomes = ['Died_within_30days_of_Discharge', 'Am_Died_within_90days_of_Discharge', 'An_Died_within_365days_of_Discharge', 
            'Readmission7d', 'Readmission14d', 'Readmission30d', 'Readmission180d']

sig_outcomes = []

for outcome in outcomes:
    outcome_1 = df_working_neg[df_working_neg[outcome] == 1]
    outcome_0 = df_working_neg[df_working_neg[outcome] == 0]
    
    # Histogram of %NT-pro change per outcome 
    hist_1 = go.Histogram(
        x=outcome_1['pc_change_72h'],
        nbinsx=50,
        name=f"{outcome} = 1",
        xaxis='x1',
        yaxis='y1'
    )
    
    hist_0 = go.Histogram(
        x=outcome_0['pc_change_72h'],
        nbinsx=50,
        name=f"{outcome} = 0",
        xaxis='x2',
        yaxis='y2'
    )
    
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.2,
        subplot_titles=[f"{outcome} = 1", f"{outcome} = 0"]
    )
    
    fig.add_trace(hist_1, row=1, col=1)
    fig.add_trace(hist_0, row=2, col=1)

    fig.update_layout(
        xaxis_title="pc_change_72h",
        yaxis_title="Frequency",
        bargap=0.2,
        xaxis=dict(range=[-100, 100]), 
        showlegend=False,
        height=600,
    )
    
    fig.show()
    
    group_1 = outcome_1['pc_change_72h']
    group_0 = outcome_0['pc_change_72h']
    
    # Mann-Whitney U Test (non-parametric) - differences between % change on outcome
    mannwhitney_u_stat, mannwhitney_u_p_value = stats.mannwhitneyu(group_1, group_0)

    print(f"Mann-Whitney U Test p-value: {mannwhitney_u_p_value}")

    if mannwhitney_u_p_value < 0.05:
        sig_outcomes.append(outcome)

In [ ]:
# Outcomes with signifiant differences between % change in NT-proBNP during admission - only those that improved
print(sig_outcomes)

In [ ]:
# Only look at outcomes with statistically significant differences - onlly improved patients

for outcome in sig_outcomes:

    outcome_1 = df_working_neg[df_working_neg[outcome] == 1]
    outcome_0 = df_working_neg[df_working_neg[outcome] == 0]

    group_1 = outcome_1['pc_change_72h']
    group_0 = outcome_0['pc_change_72h']
    
    # Median change in NT-pro per group (outcome = 1/0)
    median_1 = group_1.median()
    median_0 = group_0.median()
    
    # Mann-Whitney
    mannwhitney_u_stat, mannwhitney_u_p_value = stats.mannwhitneyu(group_1, group_0)
    
    # Effect size
    n_1 = len(group_1)
    n_0 = len(group_0)
    rank_biserial_corr = (2 * mannwhitney_u_stat) / (n_1 * n_0) - 1
    
    # Violin plots
    combined_data = pd.concat([group_1, group_0], axis=0)
    labels = [f'{outcome} = 1'] * len(group_1) + [f'{outcome} = 0'] * len(group_0)
    
    fig_violin = px.violin(
        x=combined_data,
        y=labels,
        box=True,
        points=False,
        labels={"x": "pc_change_72h", "y": f"{outcome}"},
    )
    
    print(f"Median of pc_change_72h for {outcome} = 1: {median_1}")
    print(f"Median of pc_change_72h for {outcome} = 0: {median_0}")
    print(f"Mann-Whitney U Test p-value: {mannwhitney_u_p_value}")
    print(f"Rank-biserial correlation: {rank_biserial_corr}")
    
    fig_violin.show()


In [ ]:
# NT-proBNP change by age and outcome
outcomes = ['Died_within_30days_of_Discharge', 'Am_Died_within_90days_of_Discharge', 
            'An_Died_within_365days_of_Discharge', 'Readmission7d', 'Readmission14d', 
            'Readmission30d', 'Readmission180d']


for outcome in outcomes:
    df_median = working_df_cut.groupby(['Age_on_Admission', outcome])['pc_change_72h'].median().reset_index()
    
    fig = px.line(df_median, x='Age_on_Admission', y='pc_change_72h', color=outcome,
                  title=f'Median pc_change_72h by Age on Admission and {outcome}',
                  labels={'pc_change_72h': 'Median pc_change_72h', 'Age_on_Admission': 'Age on Admission'},
                  markers=True)
    
    fig.show()


In [ ]:
# NT-proBNP by age group, grouped by outcome - Ameet defined age groups
outcomes = ['Died_within_30days_of_Discharge', 'Am_Died_within_90days_of_Discharge','An_Died_within_365days_of_Discharge', 
            'Readmission7d', 'Readmission14d', 'Readmission30d', 'Readmission180d']

age_bins = [0, 50, 65, 80, 90, 120] 
age_labels = ['0-50', '50-65', '65-80', '80-90', '90+']



working_df_cut['Age_Group'] = pd.cut(working_df_cut['Age_on_Admission'], bins=age_bins, labels=age_labels, right=False)


for outcome in outcomes:

    df_median = working_df_cut.groupby(['Age_Group', outcome])['pc_change_72h'].median().reset_index()
    

    fig = px.line(df_median, x='Age_Group', y='pc_change_72h', color=outcome,
                  title=f'Median pc_change_72h by Age on Admission and {outcome}',
                  labels={'pc_change_72h': 'Median pc_change_72h', 'Age_on_Admission': 'Age on Admission'},
                  markers=True)
    

    fig.show()


In [ ]:
outcomes = ['Died_within_30days_of_Discharge', 'Am_Died_within_90days_of_Discharge','An_Died_within_365days_of_Discharge', 
            'Readmission7d', 'Readmission14d', 'Readmission30d', 'Readmission180d']

 # Frailty onnly after 65yo, then test groups of 10
age_bins = [0, 65, 75, 85, 95, 120]  
age_labels = ['0-65', '65-75', '75-85', '85-95', '95+']


working_df_cut['Age_Group'] = pd.cut(working_df_cut['Age_on_Admission'], bins=age_bins, labels=age_labels, right=False)

for outcome in outcomes:
    df_median = working_df_cut.groupby(['Age_Group', outcome])['pc_change_72h'].median().reset_index()
    

    fig = px.line(df_median, x='Age_Group', y='pc_change_72h', color=outcome,
                  title=f'Median pc_change_72h by Age on Admission and {outcome}',
                  labels={'pc_change_72h': 'Median pc_change_72h', 'Age_on_Admission': 'Age on Admission'},
                  markers=True)
    

    fig.show()


In [ ]:
# NT-proBNP by frailty score, grouped by outcome
for outcome in outcomes:
    df_median = working_df_cut.groupby(['Frailty_Score', outcome])['pc_change_72h'].median().reset_index()
    
    fig = px.line(df_median, x='Frailty_Score', y='pc_change_72h', color=outcome,
                  title=f'Median pc_change_72h by Age on Admission and {outcome}',
                  labels={'pc_change_72h': 'Median pc_change_72h', 'Age_on_Admission': 'Age on Admission'},
                  markers=True)
    

    fig.show()
